In [ ]:
!pip install git+https://github.com/jacobgil/pytorch-grad-cam.git -q

In [ ]:
import shutil
import kagglehub
import os
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,ConfusionMatrixDisplay
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, random_split,WeightedRandomSampler , Dataset
from torchvision.datasets import ImageFolder
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Download latest version
path = kagglehub.dataset_download("salmansajid05/oral-diseases")

print("Path to dataset files:", path)

### Subtask:
Load a pre-trained YOLO model and configure it for the number of classes in the dataset.

**Reasoning**:
Import the `YOLO` class from the `ultralytics` library and load a pre-trained model, such as `yolov8n.pt`. Then, configure the model for training by specifying the path to the data configuration file (`oral_diseases.yaml`) and the number of training epochs.

### Subtask:
Create a YAML file that specifies the paths to the training and validation data and the class names.

**Reasoning**:
Create a YAML file named `oral_diseases.yaml` in the `/kaggle/working/yolo_dataset` directory. This file will contain the paths to the training and validation image directories, the number of classes, and a list of class names.

In [ ]:
#Define original directories
original_dirs = {'Calculus': '/kaggle/input/oral-diseases/Calculus/Calculus',
               'Caries' : '/kaggle/input/oral-diseases/Data caries/Data caries/caries augmented data set/preview',
               'Gingivitis' : '/kaggle/input/oral-diseases/Gingivitis/Gingivitis',
               'Mouth Ulcer' : '/kaggle/input/oral-diseases/Mouth Ulcer/Mouth Ulcer/Mouth_Ulcer_augmented_DataSet/preview',
               'Tooth Discoloration' : '/kaggle/input/oral-diseases/Tooth Discoloration/Tooth Discoloration /Tooth_discoloration_augmented_dataser/preview',
               'hypodontia' : '/kaggle/input/oral-diseases/hypodontia/hypodontia'}

#Define target base directory
base_dir = "/kaggle/working/dataset"
splits = ['train','val','test']
classes = list(original_dirs.keys())

#Create target directories
for split in splits:
    for class_name in classes :
        os.makedirs(os.path.join(base_dir,split,class_name),exist_ok = True)

#Function to copy and split images
def copy_and_transfer_images(class_name ,image_paths):
    train_path , test_path = train_test_split(image_paths , test_size = 0.1, random_state =42 )
    train_path , val_path = train_test_split(train_path , test_size = 0.15, random_state =42 )
    split_path = {'train': train_path , 'val' : val_path , 'test' : test_path}

    for split, paths in split_path.items():
        for img_path in paths :
            target_path = os.path.join(base_dir , split , class_name , os.path.basename(img_path))
            shutil.copy(img_path, target_path)

#Organize dataset
for class_name , original_dir in original_dirs.items():
    image_paths = [os.path.join(root,file) for root, _ , files in os.walk(original_dir)
                   for file in files if file.endswith(('.jpeg','.jpg','.png'))]
    if image_paths:
        copy_and_transfer_images(class_name ,image_paths)

print("Images organized successfully.")

In [ ]:
# Number of images to display per class and per row
num_images_per_class = 4
num_images_per_row = 4

# Function to fetch random images from a given directory
def get_random_images(directory , num_images):
    """
    Returns a list of random image paths from the given directory.
    Supports jpg, jpeg, and png formats.
    """
    all_images = [os.path.join(directory, image) for image in os.listdir(directory)
                  if image.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if len(all_images) <= num_images:
        return all_images

    return random.sample(all_images,num_images)

# Calculate the number of rows needed for displaying images
total_images = len(classes)*num_images_per_class
num_rows = (total_images + num_images_per_row - 1)// num_images_per_row

# Create the figure and subplots
fig, axes = plt.subplots(num_rows, num_images_per_row ,figsize = (15,num_rows*2.5) )
fig.suptitle("Random Images from Each Class", fontsize=16)

# Flatten the axes array for easy indexing
axes = axes.flatten()

# Keep track of the subplot index
image_index = 0

# Loop through each class and display random images
for class_name in classes:
    class_path = os.path.join(base_dir,'train',class_name)
    random_images = get_random_images(class_path,num_images_per_class)

    for img_path in random_images:
        if image_index <len(axes):
            image = Image.open(img_path)
            axes[image_index].imshow(image)
            axes[image_index].set_title(class_name)
            axes[image_index].axis('off')
            image_index += 1


# Hide any remaining unused subplots
for i in range(image_index, len(axes)):
    axes[i].axis('off')

# Adjust spacing and show the plot
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
#  Function to count the number of images in each split (train, val, test) for a given class
def count_images(base_dir, class_name):
    counts = {}
    for split in ["train", "val", "test"]:
        split_dir = os.path.join(base_dir, split, class_name)
        counts[split] = len(os.listdir(split_dir))
    return counts

# Count images for each class and store results in a dictionary
class_split_counts = {class_name: count_images(base_dir, class_name) for class_name in classes}

# Create subplots: 3 rows × 2 columns (total 6 plots)
fig, axes = plt.subplots(3, 2, figsize=(12, 12))
axes = axes.flatten()  # Flatten the 2D array of axes into a 1D array for easy access

# Loop through the first 6 classes and create bar plots
for idx, class_name in enumerate(classes[:6]):
    counts = class_split_counts[class_name]
    ax = axes[idx]

    # Create a bar chart for this class (Train, Val, Test)
    ax.bar(counts.keys(), counts.values(), color= ['green','yellow','red'])

    # Add titles and labels
    ax.set_title(f"{class_name}", fontsize=12)
    ax.set_xlabel("Dataset Split")
    ax.set_ylabel("Number of Images")
    ax.set_ylim(0, max(counts.values()) + 10)  # Set y-axis limit slightly above max for spacing

    # Display the count values above the bars
    for i, val in enumerate(counts.values()):
        ax.text(i, val + 1, str(val), ha="center", fontsize=10)

# Hide any unused subplots if there are fewer than 6 classes
for j in range(len(classes[:6]), len(axes)):
    axes[j].axis("off")

# Adjust layout for better visualization
plt.tight_layout()
plt.show()

In [ ]:
# Create a pie chart showing the distribution of classes in the training set
train_counts = {class_name: counts["train"] for class_name, counts in class_split_counts.items()}

plt.figure(figsize=(10, 10))
plt.pie(train_counts.values(), labels=train_counts.keys(), autopct='%1.1f%%', startangle=140)
plt.title("Distribution of Classes in Training Set")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
import pandas as pd

# Convert the dictionary to a Pandas DataFrame for easier plotting
df_counts = pd.DataFrame(class_split_counts).T
df_counts = df_counts.sort_index()

# Create a grouped bar chart
ax = df_counts.plot(kind='bar', figsize=(12, 7), rot=0)

# Add titles and labels
plt.title('Distribution of Images per Class Across Splits', fontsize=16)
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Split')
plt.tight_layout()

# Add counts on top of bars
for container in ax.containers:
    ax.bar_label(container, fmt='%d', label_type='edge', padding=3)

plt.show()

In [ ]:
# Function to display random images from each class
def display_random_images_from_classes(base_dir, classes, num_images_per_class=4):
    """
    Displays a grid of random images from each specified class in the training set.
    """
    num_classes = len(classes)
    num_rows = (num_classes * num_images_per_class + num_images_per_row - 1) // num_images_per_row

    fig, axes = plt.subplots(num_rows, num_images_per_row, figsize=(15, num_rows * 2.5))
    fig.suptitle("Random Images from Each Class (Training Set)", fontsize=16)
    axes = axes.flatten()

    img_index = 0
    for class_name in classes:
        class_path = os.path.join(base_dir, 'train', class_name)
        random_images = get_random_images(class_path, num_images_per_class)

        for img_path in random_images:
            if img_index < len(axes):
                image = Image.open(img_path)
                axes[img_index].imshow(image)
                axes[img_index].set_title(class_name)
                axes[img_index].axis('off')
                img_index += 1

    for i in range(img_index, len(axes)):
        axes[i].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Display random images from the training set
display_random_images_from_classes(base_dir, classes)

In [ ]:
# Create a pie chart showing the distribution of classes in the training set
train_counts = {class_name: counts["train"] for class_name, counts in class_split_counts.items()}

plt.figure(figsize=(10, 10))
plt.pie(train_counts.values(), labels=train_counts.keys(), autopct='%1.1f%%', startangle=140)
plt.title("Distribution of Classes in Training Set")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
#  Function to count the number of images in each split (train, val, test) for a given class
def count_images(base_dir, class_name):
    counts = {}
    for split in ["train", "val", "test"]:
        split_dir = os.path.join(base_dir, split, class_name)
        counts[split] = len(os.listdir(split_dir))
    return counts

# Count images for each class and store results in a dictionary
class_split_counts = {class_name: count_images(base_dir, class_name) for class_name in classes}

In [ ]:
# Create a pie chart showing the distribution of classes in the training set
train_counts = {class_name: counts["train"] for class_name, counts in class_split_counts.items()}

plt.figure(figsize=(10, 10))
plt.pie(train_counts.values(), labels=train_counts.keys(), autopct='%1.1f%%', startangle=140)
plt.title("Distribution of Classes in Training Set")
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
import pandas as pd

# Assuming class_split_counts is already defined from previous steps
# Convert the dictionary to a Pandas DataFrame for easier display as a table
df_counts = pd.DataFrame(class_split_counts).T

# Display the DataFrame as a table
display(df_counts)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3)) # Adjust figure size as needed
ax.axis('off') # Hide axes
ax.axis('tight') # Adjust layout

# Create the table
table = ax.table(cellText=df_counts.values,
                 colLabels=df_counts.columns,
                 rowLabels=df_counts.index,
                 loc='center')

# Style the table (optional)
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2) # Adjust scale as needed

plt.title("Distribution of Images per Class Across Splits (Table)", fontsize=14)
plt.show()

# Model Creation

In [ ]:
train_transforms = A.Compose([
    # Randomly crop and resize the image to focus on important regions like teeth and gums
    A.RandomResizedCrop(
        size =(300,300), scale=(0.95, 1.0), p=1.0
    ),

    # Randomly flip the image horizontally or vertically to simulate different viewing angles
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.1),

    # Slight shift, scaling, and rotation to simulate different head positions
    A.ShiftScaleRotate(
        shift_limit=0.05,      # up to ±5% image shift
        scale_limit=0.05,      # up to ±5% zoom in/out
        rotate_limit=15,        # max ±7 degrees rotation
        border_mode=0,         # fill empty areas with black pixels
        p=0.4
    ),

    # Adjust brightness and contrast to make infections, tissues, and gums clearer
    A.RandomBrightnessContrast(
        brightness_limit=0.15,  # ±20% brightness adjustment
        contrast_limit=0.15,   # ±25% contrast adjustment
        p=0.4
    ),

    # Apply CLAHE to locally enhance image contrast and highlight teeth/gum problems
    A.CLAHE(
        clip_limit=2.5,
        tile_grid_size=(8, 8),
        p=0.4
    ),

    # Slight color variations to simulate different lighting conditions in images
    A.HueSaturationValue(
        hue_shift_limit=8,     # small hue shift
        sat_shift_limit=20,    # adjust saturation
        val_shift_limit=10,    # adjust brightness
        p=0.4
    ),

    # Normalize image based on ImageNet statistics → suitable for InceptionV3 / EfficientNet-B3
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),

    # Convert image to PyTorch tensor format (C, H, W)
    ToTensorV2(),
], p=1.0)

# ===========================
# VALIDATION TRANSFORMATIONS
# ===========================
val_transforms = A.Compose([
    # Resize image without random cropping to keep validation images consistent
    A.Resize(height=300, width=300),

    # Normalize using the same ImageNet statistics as training
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),

    # Convert image to PyTorch tensor format
    ToTensorV2(),
], p=1.0)

In [ ]:
# Custom Dataset to integrate Albumentations
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, classes,transform=None):
        self.image_paths = image_paths
        self.classes = classes
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        img = np.array(img)

        if self.transform:
            img = self.transform(image=img)["image"]

        return img, label

In [ ]:
# Define directories
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Use ImageFolder to read image paths and labels automatically
train_data = ImageFolder(root=train_dir)
val_data = ImageFolder(root=val_dir)
test_data = ImageFolder(root=test_dir)

classes = train_data.classes

# Extract image paths and labels from ImageFolder
train_image_paths = [path for path, _ in train_data.samples]
train_labels = [label for _, label in train_data.samples]

val_image_paths = [path for path, _ in val_data.samples]
val_labels = [label for _, label in val_data.samples]

test_image_paths = [path for path, _ in test_data.samples]
test_labels = [label for _, label in test_data.samples]

# Create datasets using CustomImageDataset + Albumentations
train_dataset = CustomImageDataset(train_image_paths, train_labels,classes, transform=train_transforms)
val_dataset = CustomImageDataset(val_image_paths, val_labels,classes, transform=val_transforms)
test_dataset = CustomImageDataset(test_image_paths, test_labels,classes, transform=val_transforms)

# Class counts (your data)
class_counts = np.array([991, 1821, 990, 1943, 1402, 260])

# Compute weights for each class
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)

# Map weights to each sample in the training dataset
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
sample_weights = class_weights[train_labels_tensor]

# WeightedRandomSampler to handle class imbalance
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32 , sampler=sampler, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
def show_MRI_batch(dataloader,title = 'Batch of Images'):
    images ,labels = next(iter(dataloader))
    fig , axes = plt.subplots(4,8,figsize = (15,10))
    fig.suptitle(title)
    for i,ax in enumerate(axes.flatten()):
        if i <len(images):
            img = images[i].permute(1,2,0)
            ax.imshow(img)
            ax.set_title(train_dataset.classes[labels[i]])
            ax.axis('off')

    plt.show()

In [ ]:
show_MRI_batch(train_loader)

## Training and fitting a model

In [ ]:
class CustomEfficientnet(nn.Module):
    def __init__(self, num_classes):
        super(CustomEfficientnet,self).__init__()

        # Load pretrained ResNet50
        self.model = timm.create_model('efficientnet_b3',pretrained = True)

        # unFreeze layers
        for param in self.model.parameters():
            param.requires_grad = True

        # Replace the fully connected layer with a custom classifier
        in_features = self.model.classifier.in_features

        self.model.classifier = nn.Sequential(
            nn.Linear(in_features,1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128,num_classes)
        )

    def forward(self,x):
        return self.model(x)

In [ ]:
# Number of images per class (your data)
class_counts = torch.tensor([991, 1821, 990, 1943, 1402, 260], dtype=torch.float)

# Calculate class weights inversely proportional to class frequency
# Formula: weight = total_samples / (num_classes * class_count)
class_weights = 0
class_weights = class_counts.sum() / (len(class_counts) * class_counts)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_weights = class_weights.to(device)

print("Class names:", classes,'\n')
print("Class Weights:", class_weights,'\n')

In [ ]:
num_classes = len(train_dataset.classes)
model = CustomEfficientnet(num_classes)
model.to(device)

optimizer = optim.AdamW(model.parameters(), lr = 0.001, weight_decay = 0.00001)
criterion = nn.CrossEntropyLoss(class_weights)
scheduler = ReduceLROnPlateau(optimizer , mode = 'min' , factor = 0.2 , patience = 2)

In [ ]:
# num_epochs = 25

# scaler = torch.cuda.amp.GradScaler()

# best_model_wts = None
# best_val_loss = float("inf")

# train_loss_list = []
# train_acc_list = []
# val_loss_list = []
# val_acc_list = []


# for epoch in range(num_epochs):

#     model.train()
#     running_loss = 0.0
#     train_correct = 0
#     train_total = 0

#     for images, labels in tqdm(train_loader):
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()

#         with torch.cuda.amp.autocast():
#             outputs = model(images)
#             loss = criterion(outputs, labels)

#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()

#         running_loss += loss.item()

#         _, predicted = torch.max(outputs, 1)

#         train_total += labels.size(0)
#         train_correct += (predicted == labels).sum().item()

#     avg_train_loss = running_loss / len(train_loader)
#     train_accuracy = 100 * train_correct / train_total

#     train_loss_list.append(avg_train_loss)
#     train_acc_list.append(train_accuracy)


#     model.eval()
#     correct = 0
#     total = 0
#     val_loss = 0.0

#     all_labels = []
#     all_predictions = []

#     with torch.no_grad():
#         for images, labels in tqdm(val_loader):
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)

#             loss = criterion(outputs, labels)
#             val_loss += loss.item()

#             _, predicted = torch.max(outputs, 1)

#             all_labels.extend(labels.cpu().numpy())
#             all_predictions.extend(predicted.cpu().numpy())

#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     avg_val_loss = val_loss / len(val_loader)
#     accuracy = 100 * correct / total

#     val_loss_list.append(avg_val_loss)
#     val_acc_list.append(accuracy)

#     scheduler.step(avg_val_loss)

#     if avg_val_loss < best_val_loss:
#         best_val_loss = avg_val_loss
#         best_model_wts = model.state_dict().copy()
#         torch.save(best_model_wts, "best_model.pth")
#         print("✅ Model Saved! Best so far.")

#     print(f"Epoch [{epoch+1}/{num_epochs}] "
#       f"Train Loss: {avg_train_loss:.4f} | "
#       f"Val Loss: {avg_val_loss:.4f} | "
#       f"Train Accuracy: {train_accuracy:.2f}% | "
#       f"Val Accuracy: {accuracy:.2f}% | "
#       f"LR: {optimizer.param_groups[0]['lr']:.8f} |")

In [ ]:
# epochs = range(1, len(train_loss_list) + 1)

# plt.figure(figsize=(12,5))

# # Loss Plot
# plt.subplot(1,2,1)
# plt.plot(epochs, train_loss_list, 'b-o', label='Train Loss')
# plt.plot(epochs, val_loss_list , 'r-o', label='Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Train vs Validation Loss')
# plt.legend()

# # Accuracy Plot
# plt.subplot(1,2,2)
# plt.plot(epochs, train_acc_list, 'g-o', label='Train Accuracy')
# plt.plot(epochs, val_acc_list, 'm-o', label='Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.title('Train vs Validation Accuracy')
# plt.legend()

# plt.tight_layout()
# plt.show()

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(all_labels, all_predictions)

# Display confusion matrix with larger figure
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
fig, ax = plt.subplots(figsize=(8, 8))
# disp.plot(ax=ax, cmap="Reds")
plt.title("Confusion Matrix", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
#
print(classification_report(all_labels, all_predictions, target_names=classes))

## Model evaluation

In [ ]:
show_MRI_batch(test_loader)

In [ ]:
best_model =  CustomEfficientnet(num_classes)
best_model.load_state_dict(torch.load("best_model.pth"))
best_model.to(device)
print('best_model ready')

In [ ]:
best_model.eval()

test_labels = []
test_predictions = []

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)


        outputs = best_model(images)
        _, predicted = torch.max(outputs, 1)

        test_labels.extend(labels.cpu().numpy())
        test_predictions.extend(predicted.cpu().numpy())

        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Accuracy on Test Set: {accuracy:.2f}%")

In [ ]:
# Calculate confusion matrix
cm = confusion_matrix(test_labels, test_predictions)

# Display confusion matrix with larger figure
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
fig, ax = plt.subplots(figsize=(8, 8))
disp.plot(ax=ax, cmap="Reds")
plt.title("Confusion Matrix", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
print(classification_report(test_labels,test_predictions, target_names=classes))

In [ ]:
def visualize_grad_cam(image_paths, model, device, class_names):
    num_images = len(image_paths)
    num_rows = (num_images + 1) // 2
    plt.figure(figsize=(20, 5 * num_rows))

    # EfficientNet من timm: آخر بلوك conv
    target_layer = model.model.blocks[-1]
    grad_cam = GradCAM(model=model, target_layers=[target_layer])

    for idx, image_path in enumerate(image_paths):
        input_image = Image.open(image_path).convert('RGB')
        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
        input_tensor = preprocess(input_image).unsqueeze(0).to(device)

        grayscale_cam = grad_cam(input_tensor)[0]
        input_image_np = np.array(input_image.resize((224, 224))) / 255.0
        visualization = show_cam_on_image(input_image_np, grayscale_cam, use_rgb=True)

        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_class = class_names[predicted.item()]
        true_class = os.path.basename(os.path.dirname(image_path))
        title_color = 'green' if true_class == predicted_class else 'red'

        plt.subplot(num_rows, 4, 2 * idx + 1)
        plt.imshow(input_image_np)
        plt.title(f'True: {true_class}', fontsize=24, color=title_color)
        plt.axis('off')

        plt.subplot(num_rows, 4, 2 * idx + 2)
        plt.imshow(visualization)
        plt.title(f'Predicted: {predicted_class}', fontsize=24, color=title_color)
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
# Get random test images and visualize Grad-CAM
random_images = [os.path.join(test_dir, cls, random.choice(os.listdir(os.path.join(test_dir, cls))))
                 for cls in classes for _ in range(2)]
visualize_grad_cam(random_images, model, device, classes)

In [ ]:
# Evaluate the best Custom EfficientNet model on the test set

best_model.eval()

test_labels_clf = []
test_predictions_clf = []

correct_clf = 0
total_clf = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = best_model(images)
        _, predicted = torch.max(outputs, 1)

        test_labels_clf.extend(labels.cpu().numpy())
        test_predictions_clf.extend(predicted.cpu().numpy())

        correct_clf += (predicted == labels).sum().item()
        total_clf += labels.size(0)

accuracy_clf = 100 * correct_clf / total_clf
print(f"Accuracy of Custom EfficientNet on Test Set: {accuracy_clf:.2f}%")

# Classification Report
print("\nClassification Report for Custom EfficientNet:")
print(classification_report(test_labels_clf, test_predictions_clf, target_names=classes))

# Confusion Matrix
cm_clf = confusion_matrix(test_labels_clf, test_predictions_clf)
disp_clf = ConfusionMatrixDisplay(confusion_matrix=cm_clf, display_labels=classes)
fig_clf, ax_clf = plt.subplots(figsize=(8, 8))
disp_clf.plot(ax=ax_clf, cmap="Blues")
plt.title("Confusion Matrix for Custom EfficientNet", fontsize=16)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.show()

## Fine tuning a yolo model


# Task
Fine-tune a YOLO model using the images.

## Install ultralytics

### Subtask:
Install the necessary library for YOLO models.


**Reasoning**:
Install the ultralytics library using pip.



In [ ]:
!pip install ultralytics

## Prepare data for yolo

### Subtask:
Convert the existing image dataset into the format required by YOLO, which includes creating text files with bounding box annotations for each image.


**Reasoning**:
Create the directory structure for the YOLO dataset and then iterate through the existing dataset to copy images and create dummy label files with a single bounding box around the entire image.



In [ ]:
import os
import shutil
from PIL import Image

# Define the base directory for the YOLO dataset
yolo_base_dir = "/kaggle/working/yolo_dataset"

# Define the structure for YOLO data
yolo_data_dirs = {
    'train': os.path.join(yolo_base_dir, 'images', 'train'),
    'val': os.path.join(yolo_base_dir, 'images', 'val'),
    'test': os.path.join(yolo_base_dir, 'images', 'test'),
    'train_labels': os.path.join(yolo_base_dir, 'labels', 'train'),
    'val_labels': os.path.join(yolo_base_dir, 'labels', 'val'),
    'test_labels': os.path.join(yolo_base_dir, 'labels', 'test')
}

# Create the directories
for dir_path in yolo_data_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

# Function to create a dummy YOLO label file for an image
def create_dummy_label(image_path, label_path, class_id):
    try:
        img = Image.open(image_path)
        width, height = img.size
        # Create a dummy bounding box covering the whole image
        # Format: class_id center_x center_y width height (normalized)
        dummy_bbox = f"{class_id} 0.5 0.5 1.0 1.0"
        with open(label_path, 'w') as f:
            f.write(dummy_bbox)
    except Exception as e:
        print(f"Error creating dummy label for {image_path}: {e}")


# Copy images and create dummy labels
for split in ['train', 'val', 'test']:
    split_image_dir = yolo_data_dirs[split]
    split_label_dir = yolo_data_dirs[f'{split}_labels']

    # Use the original_dirs mapping to get the original paths
    for class_name, original_dir in original_dirs.items():
        # Find the class_id for the current class_name
        class_id = classes.index(class_name)

        # Get image paths from the original directory
        image_paths = [os.path.join(root,file) for root, _ , files in os.walk(original_dir)
                   for file in files if file.endswith(('.jpeg','.jpg','.png'))]

        # Split image paths for the current split (assuming copy_and_transfer_images did the split already)
        # If not, a splitting logic would be needed here based on the original paths
        # For simplicity, I'll assume the files are already in the 'base_dir' structure
        # that was created in the previous steps and use that as the source.
        source_dir = os.path.join(base_dir, split, class_name)
        if os.path.exists(source_dir):
            for image_name in os.listdir(source_dir):
                if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    original_image_path = os.path.join(source_dir, image_name)
                    target_image_path = os.path.join(split_image_dir, image_name)
                    label_name = os.path.splitext(image_name)[0] + '.txt'
                    target_label_path = os.path.join(split_label_dir, label_name)

                    # Copy image
                    shutil.copy(original_image_path, target_image_path)

                    # Create dummy label
                    create_dummy_label(original_image_path, target_label_path, class_id)


print("YOLO dataset structure created and dummy labels generated.")

In [ ]:
# Create the data.yaml file
data_yaml_content = f"""
path: {yolo_base_dir}
train: images/train
val: images/val
test: images/test

nc: {len(classes)}
names: {classes}
"""

with open(os.path.join(yolo_base_dir, 'oral_diseases.yaml'), 'w') as f:
    f.write(data_yaml_content)

print("YOLO configuration file 'oral_diseases.yaml' created.")

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLO model (e.g., yolov8n.pt)
model = YOLO('yolov8n.pt')

# Configure the model for training
# The data argument specifies the path to your dataset configuration file
# The epochs argument sets the number of training epochs
results = model.train(data=os.path.join(yolo_base_dir, 'oral_diseases.yaml'), epochs=25)

## Model Testing and Evaluation

### Evaluate Custom EfficientNet Model

### Evaluate YOLO Model

In [ ]:
# Run inference on a few test images using the fine-tuned YOLO model
# and visualize the predictions with bounding boxes.

# Get a few random image paths from the test set
random_test_image_paths = []
num_images_to_show = 8 # You can adjust this number
for cls in classes:
    class_path = os.path.join(test_dir, cls)
    if os.path.exists(class_path):
        images_in_class = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if images_in_class:
            random_test_image_paths.extend(random.sample(images_in_class, min(2, len(images_in_class)))) # Get up to 2 images per class

# Load the best trained YOLO model
# The weights are saved in runs/detect/train/weights/best.pt (or trainX if you ran multiple trainings)
yolo_model_path = '/content/runs/detect/train/weights/best.pt' # Update path if necessary
yolo_model = YOLO(yolo_model_path)

print("\nRunning YOLO inference on sample test images:")
yolo_results = yolo_model(random_test_image_paths)

# Visualize the results
for result in yolo_results:
    # The result object contains the image with predictions drawn on it
    display(Image.fromarray(result.plot()[:,:,::-1]))

In [ ]:
# Evaluate the YOLO model on the test set
print("\nEvaluating YOLO model on the test set:")
yolo_test_results = yolo_model.val()

print("\nYOLO Model Evaluation Metrics on Test Set:")
# The metrics are stored in the 'box' attribute of the results object
display(yolo_test_results.box.map)